#### We will be using a college dataset (College.csv) to try to classify colleges as Private or Public based off these features:
    Private: A factor with levels No and Yes indicating private or public university
    Apps: Number of applications received
    Accept: Nunber of applications accented
    Enroll: Number of new students enrotled
    Top10perc: Pct. new students from top 10% of H.S. class
    Top25perc: Pct. new students from top 25% of HS, class
    F.Undergrad: Number of fulltine undergraduetes
    P.Undergrad: Nunber of parttime undergraduates
    Outstate: Out-of-state tuition
    Roon.Board: Room and board costs
    Books: Estimated book costs
    Personal: Estimated personal spending
    PhD: Pet. of faculty with Ph.D. 's
    Terminal: Pct. of faculty with terminal degree
    S.F.Ratio: Student/faculty ratio
    perc.alumni: Pct. alumni who donate
    Expend: Instructional expenditure per student
    Grad.Rate: Graduation rate


In [ ]:
import findspark
findspark.init()

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import corr

from pyspark.ml.feature import StringIndexer, VectorIndexer, VectorAssembler, OneHotEncoder
from pyspark.ml.classification import DecisionTreeClassifier, RandomForestClassifier, GBTClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [ ]:
spark = SparkSession.builder.appName('treecode').getOrCreate()

In [ ]:
data= spark.read.csv("../../Data/College.csv", header=False, inferSchema=True)

In [ ]:
data.count()

In [ ]:
data.printSchema()

In [ ]:
data.head()

### Spark Formating of Data

In [ ]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler, StringIndexer

In [ ]:
data.columns

In [ ]:
assembler = VectorAssembler(inputCol=['Apps', 'Accept', 'Enroll', 'Top10perc', 
                             'Top25perc', 'F_Undergrad', 'P_Undergrad', 
                             'Outstate', 'Room_Board', 'Books', 'Personal', 'PhD', 
                             'Terminal', 'S_F_Ratio', 'perc_alumni', 'Expend', 'Grad_Rate'], outputCol='features')

In [ ]:
output = assembler.transform(data)

In [ ]:
# Deal with Private column being 'yes' or 'no'
indexer = StringIndexer(inputCol='Private', outputCol='PrivateIndex')
output_fixed = indexer.fit(output).transform(output)

In [ ]:
final_data = output_fixed.select('features', 'PrivateIndex')

In [ ]:
train_data, test_data = final_data.randomSplit([0.7, 0.3])

### The Classifiers

In [ ]:
# Create all 3 models
dtc = DecisionTreeClassifier(featuresCol='features', labelCol='PrivateIndex', predictionCol='prediction')
rfc = RandomForestClassifier(featuresCol='features', labelCol='PrivateIndex', predictionCol='prediction')
gbt = GBTClassifier(featuresCol='features', labelCol='PrivateIndex', predictionCol='prediction')

In [ ]:
# Train all 3 models
dtc_model = dtc.fit(train_data)
rfc_model = rfc.fit(train_data)
gbt_model = gbt.fit(train_data)

### Model Comparison

In [ ]:
dtc_predictions = dtc_model.transform(test_data)
rfc_predictions = rfc_model.transform(test_data)
gbt_predictions = gbt_model.transform(test_data)

##### Evaluation Metrics

In [ ]:
# Select (prediction, true label) and compute test error
acc_evaluator = MulticlassClassificationEvaluator(labelCol="label",
                                                  predictionCol="prediction",
                                                  metricName="accuracy")

In [ ]:
dtc_acc = acc_evaluator.evaluate(dtc_predictions)
rfc_acc = acc_evaluator.evaluate(rfc_predictions)
gbt_acc = acc_evaluator.evaluate(gbt_predictions)

In [ ]:
print("Results")
print('-'*60)
print('A single Decision Tree has an accuracy of {0:2.2f}'.format(dtc_acc*100))
print('-'*60)
print('A single Random Forest has an accuracy of {0:2.2f}'.format(rfc_acc*100))
print('-'*60)
print('An essemble using Gradient-Boosted Trees has an accuracy of {0:2.2f}'.format(gbt_acc*100))